In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import roc_auc_score

import optuna

In [ ]:
train = pd.read_csv(r'../input/tabular-playground-series-oct-2021/train.csv')
test = pd.read_csv(r'../input/tabular-playground-series-oct-2021/test.csv')
sample_submission = pd.read_csv(r'../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
train['std'] = train.std(axis=1)
train['min'] = train.min(axis=1)
train['max'] = train.max(axis=1)

test['std'] = test.std(axis=1)
test['min'] = test.min(axis=1)
test['max'] = test.max(axis=1)

In [ ]:
train.head()

Thanks to the author of this notebook https://www.kaggle.com/hamzaghanmi/tps-oct-lightgbm
I've included three more features in this.

In [ ]:
continous_cols= ['f'+str(i) for i in range(242)] + ['std']
continous_cols.remove('f22')
continous_cols.remove('f43')
categorical_cols = ['f'+str(i) for i in range(242,285)]+['f22','f43','min', 'max']
cols = continous_cols + categorical_cols

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
train[continous_cols] = scaler.fit_transform(train[continous_cols])
test[continous_cols] = scaler.transform(test[continous_cols])

In [ ]:
params={'reg_alpha': 8.158768860412389, 'reg_lambda': 8.793022151019823, 'colsample_bytree': 0.2, 'subsample': 0.4, 'learning_rate': 0.05,
   'max_depth': 100, 'num_leaves': 12, 'min_child_samples': 68, 'cat_smooth': 91,'objective': 'binary',  
            'random_state': 48,'n_estimators': 20000,'n_jobs': -1}

In [ ]:
import random
import gc
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn import preprocessing

In [ ]:
preds = np.zeros(test.shape[0])
kf = StratifiedKFold(n_splits=5,random_state=48,shuffle=True)
auc=[]  # list contains auc for each fold
n=0
for trn_idx, test_idx in kf.split(train[cols],train['target']):
    X_tr,X_val=train[cols].iloc[trn_idx],train[cols].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = LGBMClassifier(**params)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,eval_metric="auc",verbose=False)
    preds += model.predict_proba(test[cols])[:,1]/kf.n_splits
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1]))
    gc.collect()
    print(f"fold: {n+1}, auc: {auc[n]}")
    n+=1  

In [ ]:
np.mean(auc)

In [ ]:
sample_submission['target']=preds
sample_submission.to_csv('submission.csv', index=False)